## 📝 텍스트 분할기 유형

LangChain은 다양한 유형의 텍스트 분할기를 제공합니다. 이들은 모두 `langchain-text-splitters` 패키지에 포함되어 있습니다. 아래 표는 각 분할기의 몇 가지 특성과 함께 나열하고 있습니다:

| 이름 | 분할 기준 | 메타데이터 추가 | 설명 |
|----------------|----------------|----------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 재귀 | 사용자 정의 문자 목록 |  | 텍스트를 재귀적으로 분할합니다. 관련 텍스트 조각을 서로 가까이 유지하려는 목적으로 사용됩니다. 텍스트를 분할하기 시작하는 추천 방법입니다. |
| HTML | HTML 특정 문자 | ✅ | HTML 특정 문자를 기반으로 텍스트를 분할합니다. 주목할만한 점은, 청크가 어디서 왔는지에 대한 관련 정보를 추가한다는 것입니다(HTML 기반). |
| 마크다운 | 마크다운 특정 문자 | ✅ | 마크다운 특정 문자를 기반으로 텍스트를 분할합니다. 주목할만한 점은, 청크가 어디서 왔는지에 대한 관련 정보를 추가한다는 것입니다(마크다운 기반). |
| 코드 | 코드(Python, JS) 특정 문자 |  | 코딩 언어에 특정한 문자를 기반으로 텍스트를 분할합니다. 15개의 다른 언어를 선택할 수 있습니다. |
| 토큰 | 토큰 |  | 토큰을 기준으로 텍스트를 분할합니다. 토큰을 측정하는 몇 가지 다른 방법이 있습니다. |
| 문자 | 사용자 정의 문자 |  | 사용자가 정의한 문자를 기준으로 텍스트를 분할합니다. 더 단순한 방법 중 하나입니다. |
| [실험적] 의미적 청크 분할기 | 문장 |  | 먼저 문장을 기준으로 분할한 후, 옆에 있는 문장들이 의미적으로 충분히 유사한 경우 결합합니다. Greg Kamradt에서 가져옴 |
| AI21 의미적 텍스트 분할기 | 의미 | ✅ | 독립된 주제를 식별하고 이러한 주제를 따라 일관된 텍스트 조각을 분할합니다. |

### 텍스트 분할기 평가하기

Greg Kamradt가 만든 Chunkviz 유틸리티로 텍스트 분할기를 평가할 수 있습니다. Chunkviz는 텍스트 분할기가 어떻게 작동하는지 시각화하는 데 유용한 도구입니다. 텍스트가 어떻게 분할되는지 보여주며 분할 매개변수를 조정하는 데 도움을 줍니다.

### 기타 문서 변환

텍스트 분할은 LLM에 전달하기 전에 문서에서 수행하고 싶은 변환의 한 예에 불과합니다. 3rd-party 도구와의 기본 문서 변환기 통합에 대한 문서는 통합(Integrations)으로 이동하세요.


## 📑 HTML 헤더로 분할하기

### 설명 및 동기

`MarkdownHeaderTextSplitter`의 개념과 유사하게, `HTMLHeaderTextSplitter`는 "구조 인식" 청크 생성기입니다. 이 분할기는 요소 수준에서 텍스트를 나누고 각 헤더에 "관련된" 모든 청크에 대한 메타데이터를 추가합니다. 이는 요소별로 청크를 반환하거나 동일한 메타데이터를 가진 요소들을 결합할 수 있으며, 목표는 다음과 같습니다:

- (a) 문서 구조에 기반한 내용이 함께 유지되도록 관련 텍스트를 (어느 정도) 의미론적으로 그룹화하여, 구조에 따라 함께 있어야 하는 내용이 그렇게 유지되도록 합니다.
- (b) 문서 구조에 인코딩된 컨텍스트 풍부한 정보를 보존하여, 텍스트의 형식과 구성에 기반한 더 깊은 이해를 가능하게 합니다.

이 분할기는 포괄적인 청크 생성 파이프라인의 일부로 다른 텍스트 분할기와 함께 사용될 수 있으며, 구조화된 문서를 효과적으로 처리하고 분석하는 능력을 향상시킵니다.


In [2]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [4]:
'''
분할기준
첫 번째 청크: 최상위 <h1> 태그인 "Foo"를 기준으로 첫 번째 청크가 생성됩니다.
두 번째 청크: <h1> 태그 아래 있는 텍스트와 하위 헤더들까지 포함하여 "Some intro text about Foo."부터 "Bar subsection 2"까지가 하나의 큰 청크로 분할되며, 'Header 1': 'Foo'라는 메타데이터가 추가됩니다.
이후 청크들: 각 하위 섹션(<h2>, <h3>)은 자신의 내용과 관련 메타데이터(상위 헤더 정보 포함)와 함께 별도의 청크로 분할됩니다. 예를 들어, "Bar main section" 아래의 "Some intro text about Bar."는 'Header 1': 'Foo', 'Header 2': 'Bar main section' 메타데이터와 함께 독립적인 청크가 됩니다.
'''

from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Foo</h1>
        <p>Some intro text about Foo.</p>
        <div>
            <h2>Bar main section</h2>
            <p>Some intro text about Bar.</p>
            <h3>Bar subsection 1</h3>
            <p>Some text about the first subtopic of Bar.</p>
            <h3>Bar subsection 2</h3>
            <p>Some text about the second subtopic of Bar.</p>
        </div>
        <div>
            <h2>Baz</h2>
            <p>Some text about Baz</p>
        </div>
        <br>
        <p>Some concluding text about Foo</p>
    </div>
</body>
</html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

'\n분할기준\n첫 번째 청크: 최상위 <h1> 태그인 "Foo"를 기준으로 첫 번째 청크가 생성됩니다.\n두 번째 청크: <h1> 태그 아래 있는 텍스트와 하위 헤더들까지 포함하여 "Some intro text about Foo."부터 "Bar subsection 2"까지가 하나의 큰 청크로 분할되며, \'Header 1\': \'Foo\'라는 메타데이터가 추가됩니다.\n이후 청크들: 각 하위 섹션(<h2>, <h3>)은 자신의 내용과 관련 메타데이터(상위 헤더 정보 포함)와 함께 별도의 청크로 분할됩니다. 예를 들어, "Bar main section" 아래의 "Some intro text about Bar."는 \'Header 1\': \'Foo\', \'Header 2\': \'Bar main section\' 메타데이터와 함께 독립적인 청크가 됩니다.\n'

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import HTMLHeaderTextSplitter
url = "https://plato.stanford.edu/entries/goedel/"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text_from_url(url)

chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(html_header_splits)
splits[80:85]

[Document(page_content='We see that Gödel first tried to reduce the consistency problem for analysis to that of arithmetic. This seemed to require a truth definition for arithmetic, which in turn led to paradoxes, such as the Liar paradox (“This sentence is false”) and Berry’s paradox (“The least number not defined by an expression consisting of just fourteen English words”). Gödel then noticed that such paradoxes would not necessarily arise if truth were replaced by provability. But this means that arithmetic truth', metadata={'Header 1': 'Kurt Gödel', 'Header 2': '2. Gödel’s Mathematical Work', 'Header 3': '2.2 The Incompleteness Theorems', 'Header 4': '2.2.1 The First Incompleteness Theorem'}),
 Document(page_content='means that arithmetic truth and arithmetic provability are not co-extensive — whence the First Incompleteness Theorem.', metadata={'Header 1': 'Kurt Gödel', 'Header 2': '2. Gödel’s Mathematical Work', 'Header 3': '2.2 The Incompleteness Theorems', 'Header 4': '2.2.1 Th

## 📑 HTML 섹션으로 분할하기

### 설명 및 동기

`HTMLHeaderTextSplitter`의 개념과 유사하게, `HTMLSectionSplitter`는 "구조 인식" 청크 생성기입니다. 이 분할기는 요소 수준에서 텍스트를 나누고, 각 헤더에 "관련된" 모든 청크에 대한 메타데이터를 추가합니다. 이는 요소별로 청크를 반환하거나 동일한 메타데이터를 가진 요소들을 결합할 수 있으며, 목표는 다음과 같습니다:

- (a) 문서의 구조에 기반하여 관련 있는 텍스트를 (어느 정도) 의미론적으로 그룹화하여, 문서의 읽기 흐름을 유지합니다.
- (b) 문서의 구조에서 인코딩된 컨텍스트 정보를 보존하여, 각 청크가 문서 내에서 어떤 위치와 관련 있는지를 명확하게 합니다.

이 분할기는 청크 파이프라인의 일부로 다른 텍스트 분할기와 함께 사용될 수 있습니다. 내부적으로는 섹션 크기가 청크 크기보다 클 경우 `RecursiveCharacterTextSplitter`를 사용합니다. 또한, 결정된 폰트 크기 임계값을 기반으로 텍스트가 섹션인지 여부를 판단하기 위해 텍스트의 폰트 크기를 고려합니다. `xslt_path`를 사용하여 HTML을 변환하여 제공된 태그를 기반으로 섹션을 감지할 수 있도록 절대 경로를 제공합니다. 기본값은 `data_connection/document_transformers` 디렉토리의 `converting_to_header.xslt` 파일을 사용하는 것입니다. 이는 HTML을 섹션을 감지하기 쉬운 형식/레이아웃으로 변환하기 위한 것입니다. 예를 들어, 폰트 크기를 기반으로 한 `span`을 헤더 태그로 변환하여 섹션으로 감지될 수 있게 합니다.


In [1]:
from langchain_text_splitters import HTMLSectionSplitter

html_string = """
    <!DOCTYPE html>
    <html>
    <body>
        <div>
            <h1>Foo</h1>
            <p>Some intro text about Foo.</p>
            <div>
                <h2>Bar main section</h2>
                <p>Some intro text about Bar.</p>
                <h3>Bar subsection 1</h3>
                <p>Some text about the first subtopic of Bar.</p>
                <h3>Bar subsection 2</h3>
                <p>Some text about the second subtopic of Bar.</p>
            </div>
            <div>
                <h2>Baz</h2>
                <p>Some text about Baz</p>
            </div>
            <br>
            <p>Some concluding text about Foo</p>
        </div>
    </body>
    </html>
"""

headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2")]

html_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

ImportError: cannot import name 'HTMLSectionSplitter' from 'langchain_text_splitters' (/Users/kdb/anaconda3/envs/kdb/lib/python3.9/site-packages/langchain_text_splitters/__init__.py)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

html_string = """
    <!DOCTYPE html>
    <html>
    <body>
        <div>
            <h1>Foo</h1>
            <p>Some intro text about Foo.</p>
            <div>
                <h2>Bar main section</h2>
                <p>Some intro text about Bar.</p>
                <h3>Bar subsection 1</h3>
                <p>Some text about the first subtopic of Bar.</p>
                <h3>Bar subsection 2</h3>
                <p>Some text about the second subtopic of Bar.</p>
            </div>
            <div>
                <h2>Baz</h2>
                <p>Some text about Baz</p>
            </div>
            <br>
            <p>Some concluding text about Foo</p>
        </div>
    </body>
    </html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

html_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)

html_header_splits = html_splitter.split_text(html_string)

chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(html_header_splits)
splits

# Split by character
- 가장 간단한 방법입니다. 문자(기본값은 "")를 기준으로 분할하고 청크 길이를 문자 수로 측정합니다.
- 텍스트 분할 방식: 단일 문자 기준.
- 청크 크기를 측정하는 방법: 문자 수로 측정합니다.

In [20]:
with open("/Users/kdb/Desktop/psedo_labs/4weeks_retrieval/data/datas.txt") as f:
    state_of_the_union = f.read()

In [21]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=10,
    chunk_overlap=2,
    length_function=len,
    is_separator_regex=False,
)

In [22]:
texts = text_splitter.create_documents([state_of_the_union])
print(texts[0])

Created a chunk of size 181, which is longer than the specified 10
Created a chunk of size 89, which is longer than the specified 10
Created a chunk of size 190, which is longer than the specified 10
Created a chunk of size 120, which is longer than the specified 10
Created a chunk of size 22, which is longer than the specified 10
Created a chunk of size 218, which is longer than the specified 10
Created a chunk of size 212, which is longer than the specified 10
Created a chunk of size 199, which is longer than the specified 10
Created a chunk of size 137, which is longer than the specified 10
Created a chunk of size 32, which is longer than the specified 10
Created a chunk of size 149, which is longer than the specified 10


page_content="sonny sonny \n kdb kdb🐱 \n holland😻😻\n 손흥민(토트넘)이 만능 공격수의 상징인 '10골 10도움(10-10)'의 세 번째 달성에 1도움만을 남겨두면서 잉글랜드 프리미어리그(EPL) 첼시의 레전드 디디에 드록바와 어깨를 나란히 할 날이 머지않았다. 드록바의 EPL 통산 골기록은 이미 지난해 넘어섰다."


다음은 문서와 함께 메타데이터를 전달하는 예시이며, 문서와 함께 분할되어 있는 것을 확인할 수 있습니다.

In [23]:
metadatas = [{"document": 1}, {"document": 2}]
documents = text_splitter.create_documents(
    [state_of_the_union, state_of_the_union], metadatas=metadatas
)
print(documents[0])

Created a chunk of size 181, which is longer than the specified 10
Created a chunk of size 89, which is longer than the specified 10
Created a chunk of size 190, which is longer than the specified 10
Created a chunk of size 120, which is longer than the specified 10
Created a chunk of size 22, which is longer than the specified 10
Created a chunk of size 218, which is longer than the specified 10
Created a chunk of size 212, which is longer than the specified 10
Created a chunk of size 199, which is longer than the specified 10
Created a chunk of size 137, which is longer than the specified 10
Created a chunk of size 32, which is longer than the specified 10
Created a chunk of size 149, which is longer than the specified 10
Created a chunk of size 181, which is longer than the specified 10
Created a chunk of size 89, which is longer than the specified 10
Created a chunk of size 190, which is longer than the specified 10
Created a chunk of size 120, which is longer than the specified 10

page_content="sonny sonny \n kdb kdb🐱 \n holland😻😻\n 손흥민(토트넘)이 만능 공격수의 상징인 '10골 10도움(10-10)'의 세 번째 달성에 1도움만을 남겨두면서 잉글랜드 프리미어리그(EPL) 첼시의 레전드 디디에 드록바와 어깨를 나란히 할 날이 머지않았다. 드록바의 EPL 통산 골기록은 이미 지난해 넘어섰다." metadata={'document': 1}


In [24]:
text_splitter.split_text(state_of_the_union)[0]

Created a chunk of size 181, which is longer than the specified 10
Created a chunk of size 89, which is longer than the specified 10
Created a chunk of size 190, which is longer than the specified 10
Created a chunk of size 120, which is longer than the specified 10
Created a chunk of size 22, which is longer than the specified 10
Created a chunk of size 218, which is longer than the specified 10
Created a chunk of size 212, which is longer than the specified 10
Created a chunk of size 199, which is longer than the specified 10
Created a chunk of size 137, which is longer than the specified 10
Created a chunk of size 32, which is longer than the specified 10
Created a chunk of size 149, which is longer than the specified 10


"sonny sonny \n kdb kdb🐱 \n holland😻😻\n 손흥민(토트넘)이 만능 공격수의 상징인 '10골 10도움(10-10)'의 세 번째 달성에 1도움만을 남겨두면서 잉글랜드 프리미어리그(EPL) 첼시의 레전드 디디에 드록바와 어깨를 나란히 할 날이 머지않았다. 드록바의 EPL 통산 골기록은 이미 지난해 넘어섰다."

# Split code

CodeTextSplitter를 사용하면 여러 언어를 지원하여 코드를 분할할 수 있습니다. 열거형 언어를 가져와서 언어를 지정합니다.

In [27]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
# 지원하는 언어 목록
[e.value for e in Language]

['cpp',
 'go',
 'java',
 'kotlin',
 'js',
 'ts',
 'php',
 'proto',
 'python',
 'rst',
 'ruby',
 'rust',
 'scala',
 'swift',
 'markdown',
 'latex',
 'html',
 'sol',
 'csharp',
 'cobol',
 'c',
 'lua',
 'perl']

In [30]:
# 언어별 구분자 
RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON)

['\nclass ', '\ndef ', '\n\tdef ', '\n\n', '\n', ' ', '']

In [31]:
#python
PYTHON_CODE = """
def hello_world():
    print("Hello, World!")

# Call the function
hello_world()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(page_content='def hello_world():\n    print("Hello, World!")'),
 Document(page_content='# Call the function\nhello_world()')]

In [32]:
#javascript
JS_CODE = """
function helloWorld() {
  console.log("Hello, World!");
}

// Call the function
helloWorld();
"""

js_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JS, chunk_size=60, chunk_overlap=0
)
js_docs = js_splitter.create_documents([JS_CODE])
js_docs

[Document(page_content='function helloWorld() {\n  console.log("Hello, World!");\n}'),
 Document(page_content='// Call the function\nhelloWorld();')]

In [33]:
#markdown
markdown_text = """
# 🦜️🔗 LangChain

⚡ Building applications with LLMs through composability ⚡

## Quick Install

```bash
# Hopefully this code block isn't split
pip install langchain
```

As an open-source project in a rapidly developing field, we are extremely open to contributions.
"""

In [34]:
md_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN, chunk_size=60, chunk_overlap=0
)
md_docs = md_splitter.create_documents([markdown_text])
md_docs

[Document(page_content='# 🦜️🔗 LangChain'),
 Document(page_content='⚡ Building applications with LLMs through composability ⚡'),
 Document(page_content='## Quick Install\n\n```bash'),
 Document(page_content="# Hopefully this code block isn't split"),
 Document(page_content='pip install langchain'),
 Document(page_content='```'),
 Document(page_content='As an open-source project in a rapidly developing field, we'),
 Document(page_content='are extremely open to contributions.')]

In [35]:
#C
C_CODE = """
using System;
class Program
{
    static void Main()
    {
        int age = 30; // Change the age value as needed

        // Categorize the age without any console output
        if (age < 18)
        {
            // Age is under 18
        }
        else if (age >= 18 && age < 65)
        {
            // Age is an adult
        }
        else
        {
            // Age is a senior citizen
        }
    }
}
"""
c_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CSHARP, chunk_size=128, chunk_overlap=0
)
c_docs = c_splitter.create_documents([C_CODE])
c_docs

[Document(page_content='using System;'),
 Document(page_content='class Program\n{\n    static void Main()\n    {\n        int age = 30; // Change the age value as needed'),
 Document(page_content='// Categorize the age without any console output\n        if (age < 18)\n        {\n            // Age is under 18'),
 Document(page_content='}\n        else if (age >= 18 && age < 65)\n        {\n            // Age is an adult\n        }\n        else\n        {'),
 Document(page_content='// Age is a senior citizen\n        }\n    }\n}')]

## MarkdownHeaderTextSplitter
### 동기
많은 채팅이나 Q+A 애플리케이션에서는 입력 문서를 임베딩하고 벡터 저장 전에 청크로 분할하는 작업이 포함됩니다.

Pinecone의 이러한 노트들은 몇 가지 유용한 팁을 제공합니다:

- 전체 문단이나 문서가 임베딩될 때, 임베딩 과정은 전반적인 맥락과 텍스트 내의 문장과 구절 간의 관계를 모두 고려합니다. 이는 텍스트의 더 넓은 의미와 주제를 포착하는 더 포괄적인 벡터 표현을 결과로 낼 수 있습니다.

앞서 언급했듯이, 청크 분할의 주요 목표는 공통 맥락을 가진 텍스트를 함께 유지하는 것입니다. 이를 염두에 두고, 문서 자체의 구조를 특별히 존중하고자 할 수 있습니다. 예를 들어, 마크다운 파일은 헤더에 의해 구성됩니다. 특정 헤더 그룹 내에서 청크를 생성하는 것은 직관적인 아이디어입니다. 이 도전을 해결하기 위해, 우리는 `MarkdownHeaderTextSplitter`를 사용할 수 있습니다. 이는 지정된 헤더 세트에 따라 마크다운 파일을 분할합니다.





In [36]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [37]:
markdown_document = "# Foo\n\n    ## Bar\n\nHi this is Jim\n\nHi this is Joe\n\n ### Boo \n\n Hi this is Lance \n\n ## Baz\n\n Hi this is Molly"

headers_to_split_on = [
    ("#", "Header 1"), # 첫번째 헤더기준으로 분할
    ("##", "Header 2"), # 두번째 헤더기준으로 분할
    ("###", "Header 3"), # 세번째 헤더기준으로 분할
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_document)
md_header_splits

[Document(page_content='Hi this is Jim  \nHi this is Joe', metadata={'Header 1': 'Foo', 'Header 2': 'Bar'}),
 Document(page_content='Hi this is Lance', metadata={'Header 1': 'Foo', 'Header 2': 'Bar', 'Header 3': 'Boo'}),
 Document(page_content='Hi this is Molly', metadata={'Header 1': 'Foo', 'Header 2': 'Baz'})]

In [38]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)
md_header_splits

[Document(page_content='# Foo  \n## Bar  \nHi this is Jim  \nHi this is Joe', metadata={'Header 1': 'Foo', 'Header 2': 'Bar'}),
 Document(page_content='### Boo  \nHi this is Lance', metadata={'Header 1': 'Foo', 'Header 2': 'Bar', 'Header 3': 'Boo'}),
 Document(page_content='## Baz  \nHi this is Molly', metadata={'Header 1': 'Foo', 'Header 2': 'Baz'})]

In [39]:
markdown_document = "# Intro \n\n    ## History \n\n Markdown[9] is a lightweight markup language for creating formatted text using a plain-text editor. John Gruber created Markdown in 2004 as a markup language that is appealing to human readers in its source code form.[9] \n\n Markdown is widely used in blogging, instant messaging, online forums, collaborative software, documentation pages, and readme files. \n\n ## Rise and divergence \n\n As Markdown popularity grew rapidly, many Markdown implementations appeared, driven mostly by the need for \n\n additional features such as tables, footnotes, definition lists,[note 1] and Markdown inside HTML blocks. \n\n #### Standardization \n\n From 2012, a group of people, including Jeff Atwood and John MacFarlane, launched what Atwood characterised as a standardisation effort. \n\n ## Implementations \n\n Implementations of Markdown are available for over a dozen programming languages."

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]

# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)

# Char-level splits
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 250
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(md_header_splits)
splits

[Document(page_content='# Intro  \n## History  \nMarkdown[9] is a lightweight markup language for creating formatted text using a plain-text editor. John Gruber created Markdown in 2004 as a markup language that is appealing to human readers in its source code form.[9]', metadata={'Header 1': 'Intro', 'Header 2': 'History'}),
 Document(page_content='Markdown is widely used in blogging, instant messaging, online forums, collaborative software, documentation pages, and readme files.', metadata={'Header 1': 'Intro', 'Header 2': 'History'}),
 Document(page_content='## Rise and divergence  \nAs Markdown popularity grew rapidly, many Markdown implementations appeared, driven mostly by the need for  \nadditional features such as tables, footnotes, definition lists,[note 1] and Markdown inside HTML blocks.', metadata={'Header 1': 'Intro', 'Header 2': 'Rise and divergence'}),
 Document(page_content='#### Standardization  \nFrom 2012, a group of people, including Jeff Atwood and John MacFarlane,

## JSON 재귀적으로 분할하기

이 JSON 분할기는 JSON 데이터를 깊이 우선으로 탐색하며 더 작은 JSON 청크를 생성합니다. 이는 중첩된 JSON 객체를 가능한 한 전체로 유지하려 하지만, 청크를 최소 청크 크기(`min_chunk_size`)와 최대 청크 크기(`max_chunk_size`) 사이로 유지하기 위해 필요한 경우 분할합니다. 값이 중첩된 JSON이 아니라 매우 큰 문자열인 경우, 문자열은 분할되지 않습니다. 청크 크기에 엄격한 상한선이 필요한 경우, 이러한 청크에 대해 `Recursive Text splitter`를 사용하는 것을 고려하십시오. 리스트를 분할하는 선택적인 전처리 단계가 있는데, 이는 먼저 리스트를 JSON(사전)으로 변환한 다음 이를 해당하는 방식으로 분할하는 것입니다.

### 텍스트 분할 방식
- JSON 값으로 분할됩니다.

### 청크 크기 측정 방식
- 문자 수에 의해 측정됩니다.


In [40]:
import json

import requests

# This is a large nested json object and will be loaded as a python dict
json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()

from langchain_text_splitters import RecursiveJsonSplitter

splitter = RecursiveJsonSplitter(max_chunk_size=300)

# Recursively split json data - If you need to access/manipulate the smaller json chunks
json_chunks = splitter.split_json(json_data=json_data)

# The splitter can also output documents
docs = splitter.create_documents(texts=[json_data])

# or a list of strings
texts = splitter.split_text(json_data=json_data)

print(texts[0])
print(texts[1])

{"openapi": "3.1.0", "info": {"title": "LangSmith", "version": "0.1.0"}, "servers": [{"url": "https://api.smith.langchain.com", "description": "LangSmith API endpoint."}]}
{"paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": ["tracer-sessions"], "summary": "Read Tracer Session", "description": "Get a specific session.", "operationId": "read_tracer_session_api_v1_sessions__session_id__get"}}}}


In [41]:
docs

[Document(page_content='{"openapi": "3.1.0", "info": {"title": "LangSmith", "version": "0.1.0"}, "servers": [{"url": "https://api.smith.langchain.com", "description": "LangSmith API endpoint."}]}'),
 Document(page_content='{"paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": ["tracer-sessions"], "summary": "Read Tracer Session", "description": "Get a specific session.", "operationId": "read_tracer_session_api_v1_sessions__session_id__get"}}}}'),
 Document(page_content='{"paths": {"/api/v1/sessions/{session_id}": {"get": {"security": [{"API Key": []}, {"Tenant ID": []}, {"Bearer Auth": []}]}}}}'),
 Document(page_content='{"paths": {"/api/v1/sessions/{session_id}": {"get": {"parameters": [{"name": "session_id", "in": "path", "required": true, "schema": {"type": "string", "format": "uuid", "title": "Session Id"}}, {"name": "include_stats", "in": "query", "required": false, "schema": {"type": "boolean", "default": false, "title": "Include Stats"}}, {"name": "accept", "in": "header"

In [43]:
texts

['{"openapi": "3.1.0", "info": {"title": "LangSmith", "version": "0.1.0"}, "servers": [{"url": "https://api.smith.langchain.com", "description": "LangSmith API endpoint."}]}',
 '{"paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": ["tracer-sessions"], "summary": "Read Tracer Session", "description": "Get a specific session.", "operationId": "read_tracer_session_api_v1_sessions__session_id__get"}}}}',
 '{"paths": {"/api/v1/sessions/{session_id}": {"get": {"security": [{"API Key": []}, {"Tenant ID": []}, {"Bearer Auth": []}]}}}}',
 '{"paths": {"/api/v1/sessions/{session_id}": {"get": {"parameters": [{"name": "session_id", "in": "path", "required": true, "schema": {"type": "string", "format": "uuid", "title": "Session Id"}}, {"name": "include_stats", "in": "query", "required": false, "schema": {"type": "boolean", "default": false, "title": "Include Stats"}}, {"name": "accept", "in": "header", "required": false, "schema": {"anyOf": [{"type": "string"}, {"type": "null"}], "title": "

In [44]:
# Let's look at the size of the chunks
print([len(text) for text in texts][:10])

# Reviewing one of these chunks that was bigger we see there is a list object there
print(texts[1])

[171, 231, 126, 469, 210, 213, 237, 271, 191, 232]
{"paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": ["tracer-sessions"], "summary": "Read Tracer Session", "description": "Get a specific session.", "operationId": "read_tracer_session_api_v1_sessions__session_id__get"}}}}


In [45]:
# 기본적으로 json 스플리터는 목록을 분할하지 않습니다.
# 다음은 json을 전처리하고 index:item을 key:val 쌍으로 사용하여 목록을 딕셔너리로 변환합니다.
texts = splitter.split_text(json_data=json_data, convert_lists=True)

In [46]:
print([len(text) for text in texts][:10])

[171, 231, 126, 469, 210, 213, 237, 271, 191, 232]


## 📖 문자별 재귀적 분할

이 텍스트 분할기는 일반 텍스트에 대해 권장되는 분할기입니다. 문자의 리스트에 의해 파라미터화되며, 청크가 충분히 작아질 때까지 순서대로 그 문자들에 따라 분할을 시도합니다. 기본 문자 리스트는 `["\n\n", "\n", " ", ""]` 입니다. 이는 가능한 한 오래 문단(그 다음 문장, 그리고 단어)을 함께 유지하려는 효과를 가지며, 이는 일반적으로 의미론적으로 가장 강하게 연관된 텍스트 조각으로 보입니다.

### 텍스트 분할 방식
- 문자 리스트에 의해 분할됩니다.

### 청크 크기 측정 방식
- 문자 수에 의해 측정됩니다.


In [ ]:
# This is a long document we can split up.
with open("../../state_of_the_union.txt") as f:
    state_of_the_union = f.read()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents([state_of_the_union])
print(texts[0])
print(texts[1])

### Splitting text from languages without word boundaries
일부 언어 시스템에서 단어 경계가 명확히 정의되지 않는 언어(예: 중국어, 일본어, 태국어)의 텍스트를 분할할 때의 처리 방법을 설명합니다. 기본 분리자 리스트(["\n\n", "\n", " ", ""])를 사용하면, 이러한 언어에서 단어가 청크 사이에 나뉠 수 있습니다. 단어들이 함께 유지되도록 하기 위해서, 추가적인 구두점을 포함하여 분리자 리스트를 오버라이드할 수 있습니다.

추가된 분리자:
ASCII 온점("."), 유니코드 전각 온점("．") (중국어 텍스트에서 사용됨), 한자식 온점("。") (일본어 및 중국어에서 사용됨)을 추가합니다.
태국어, 미얀마어, 크메르어, 일본어에서 사용되는 영폭 공백을 추가합니다.
**ASCII 쉼표(","), 유니코드 전각 쉼표("，"), 유니코드 한자식 쉼표("、")**를 추가합니다.
코드 설명:
RecursiveCharacterTextSplitter 인스턴스를 생성할 때 separators 인자에 이러한 추가 분리자들을 포함시킵니다. 이는 기본적인 공백과 개행 문자 외에도, 주어진 언어에 맞게 텍스트를 더 적절히 분할할 수 있게 해 줍니다. 즉, 이 방법을 통해 언어의 특성을 고려하여 단어가 청크 사이에 나눠지는 것을 방지하고, 가능한 한 관련된 텍스트를 함께 유지할 수 있습니다.

이 접근 방식은 텍스트 분석이나 처리 작업에서 문맥을 보다 잘 유지하고자 할 때 특히 유용하며, 단어 경계가 명확하지 않은 언어에 대한 처리를 개선합니다.






In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200B",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    # Existing args
)

## 🧠 의미론적 청크 생성 Semantic Chunking

In [1]:
# This is a long document we can split up.
with open("/Users/kdb/Desktop/psedo_labs/4weeks_retrieval/data/datas.txt") as f: 
    state_of_the_union = f.read()

In [2]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(OpenAIEmbeddings(api_key='your_api_key'))#임베딩 성능이 좋으면 좋을수록 의미에 관련된 청크가 더 잘 분할됩니다.

docs = text_splitter.create_documents([state_of_the_union])
print(docs[0].page_content)

ModuleNotFoundError: No module named 'langchain_experimental'

### Breakpoins
이 청커는 문장을 "분리"할 시점을 결정하는 방식으로 작동합니다. 이 작업은 두 문장 사이의 임베딩 차이를 찾아서 수행됩니다. 그 차이가 임계값을 넘으면 문장이 분리됩니다.
임계값을 결정하는 몇 가지 방법이 있습니다.


### 1. 백분위수 (Percentile)
- 기본적으로 텍스트는 백분위수에 기반하여 분할됩니다. 이 방법에서는 모든 문장 간의 차이를 계산하고, X 백분위수보다 큰 차이를 가진 문장을 분할합니다.
```python
text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="percentile"
)
```
### 2. 표준 편차 (Standard Deviation)
표준 편차 방법에서는 X 표준 편차보다 큰 차이를 가진 문장을 분할합니다.
```python
text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="standard_deviation"
)
```
### 3. 사분위수 간 거리 (Interquartile)
사분위수 간 거리 방법에서는 사분위수 간 거리를 사용하여 청크를 분할합니다.
```python
text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="interquartile"
)
```

# Split by tokens

### 🚀 토큰 제한과 텍스트 분할

언어 모델은 토큰 제한을 가지고 있습니다. 토큰 제한을 초과해서는 안 됩니다. 따라서 텍스트를 청크로 분할할 때 토큰 수를 세는 것이 좋은 생각입니다. 많은 토크나이저가 있지만, 텍스트에서 토큰을 세려면 언어 모델에서 사용되는 것과 동일한 토크나이저를 사용해야 합니다.

#### 📖 TikToken
TikToken은 OpenAI에 의해 만들어진 빠른 BPE(Bite Pair Encoding) 토크나이저입니다.

이를 사용하여 사용된 토큰을 추정할 수 있으며, OpenAI 모델에 대해서는 더 정확할 것입니다.

#### 텍스트 분할 방식
- 전달된 문자에 의해 분할됩니다.

#### 청크 크기 측정 방식
- TikToken 토크나이저에 의해 측정됩니다.

# tiktoken 


.from_tiktoken_encoder() 메서드는 인코딩(예: cl100k_base) 또는 모델명(예: gpt-4)을 인수로 받습니다. chunk_size, chunk_overlap, 구분 기호 같은 모든 추가 인수는 CharacterTextSplitter를 인스턴스화하는 데 사용됩니다:

In [ ]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding="cl100k_base", chunk_size=100, chunk_overlap=0 #100개의 토큰으로 분할
)
texts = text_splitter.split_text(state_of_the_union)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder( 
    model_name="gpt-4",
    chunk_size=100,
    chunk_overlap=0,
)

In [ ]:
from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)

texts = text_splitter.split_text(state_of_the_union)
print(texts[0])

# Hugging Face Tokenizer

Hugging Face는 다양한 토크나이저를 제공합니다.

우리는 Hugging Face의 `GPT2TokenizerFast`를 사용하여 텍스트 길이를 토큰 수로 계산합니다.

### 텍스트 분할 방식
- 전달된 문자에 의해 분할됩니다.

### 청크 크기 측정 방식
- Hugging Face 토크나이저에 의해 계산된 토큰 수에 의해 측정됩니다.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# 분할할 긴 문서입니다.
with open("../../../state_of_the_union.txt") as f:
    state_of_the_union = f.read()


text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=100, chunk_overlap=0
)
texts = text_splitter.split_text(state_of_the_union)

print(texts[0])